### Import the libraries

In [25]:
# Import the required libraries
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from pandas import Timestamp

with open("bmodel.pkl", "rb") as file:
    bmodel = pickle.load(file)
with open("emodel.pkl", "rb") as file:
    emodel = pickle.load(file)
with open("lmodel.pkl", "rb") as file:
    lmodel = pickle.load(file)

### Define the Functions

In [61]:
def predict_future_price(crypto_type, model_fit, train, date):
    model_pred = []
    predictions = []

#     for date in future_dates:
    train_data = train.copy()  # Create a copy of the training data for each iteration

    # Make prediction for the future date
    forecast = model_fit.forecast(steps=1)
    prediction = forecast[0]
    train_data.append(prediction)  # Update the copy of the training data with the predicted value

    # Retrain the model with the updated training data
    model = sm.tsa.arima.ARIMA(train_data, order=(4, 1, 0))
    model_fit = model.fit()
    model_pred.append(prediction)
    predictions.append((date, prediction))

    # Create a DataFrame for side-by-side comparison
    df_predictions = pd.DataFrame(predictions, columns=['Date', 'Prediction'])
    df_predictions.set_index('Date', inplace=True)

    return model_pred, df_predictions, model_fit

In [62]:
# Function to simulate trading decisions
def simulate_trading(predictions, latest):
    # Define trading logic based on predictions
    trading_decision = None
    price_difference = predictions - latest
    if abs(price_difference) <= 50:
        trading_decision = "Hold"
    elif price_difference > 50:
        trading_decision = "Sell"
    else:
        trading_decision = "Buy"
    return trading_decision

In [63]:
# Main app code
def main(Date, Coin):
    with open("bmodel.pkl", "rb") as file:
        bmodel = pickle.load(file)
    with open("emodel.pkl", "rb") as file:
        emodel = pickle.load(file)
    with open("lmodel.pkl", "rb") as file:
        lmodel = pickle.load(file)
    df= pd.read_csv('crypto.csv', index_col=False)
    
    ##Bitcoin
    bdf = df[df['Coin'] == 'Bitcoin']
    bdf = bdf.set_index('Date')
    bdf= bdf['Close']
    b = int(len(bdf) * 0.9)
    btrain = list(bdf[0:b])
    btest = list(bdf[b:])

    ##ethereum
    edf = df[df['Coin'] == 'Ethereum']
    edf = edf.set_index('Date')
    edf= edf['Close']
    e = int(len(edf) * 0.9)
    etrain = list(edf[0:e])
    etest = list(edf[e:])

    ##Litecoin
    ldf = df[df['Coin'] == 'Litecoin']
    ldf = ldf.set_index('Date')
    ldf= ldf['Close']
    l = int(len(ldf) * 0.9)
    ltrain = list(ldf[0:l])
    ltest = list(ldf[l:])
  
    if Coin == 'Bitcoin':
        train = btrain
        model_fit = bmodel
        predictions, df_predictions, bmodel = predict_future_price(Coin, model_fit, train, Date)
        latest = bdf[-1]
        with open("bmodel.pkl", "wb") as file:
            pickle.dump(bmodel, file)
    elif Coin == 'Ethereum':
        train = etrain
        model_fit = emodel
        predictions, df_predictions, emodel = predict_future_price(Coin, model_fit, train, Date)
        latest = edf[-1]
        with open("emodel.pkl", "wb") as file:
            pickle.dump(emodel, file)

    elif Coin == 'Litecoin':
        train = ltrain
        model_fit = lmodel
        predictions, df_predictions, lmodel = predict_future_price(Coin, model_fit, train, Date)
        latest = ldf[-1]
        with open("lmodel.pkl", "wb") as file:
            pickle.dump(lmodel, file)
    else:
        raise ValueError("Invalid cryptocurrency type. Choose one from 'Bitcoin', 'Ethereum', or 'Litecoin'.")


    trading_decisions = simulate_trading(predictions[0], latest )

    return predictions[0], trading_decisions





### Data

In [65]:
Date = input("Enter the future date (YYYY-MM-DD): ")
Coin =  input("Enter the cryptocurrency type (bitcoin, ethereum, litecoin): ")



Enter the future date (YYYY-MM-DD): 2025-12-15
Enter the cryptocurrency type (bitcoin, ethereum, litecoin): Ethereum


### Output

In [66]:
print(main(Date, Coin))

(1756.3131242088764, 'Buy')


## Save the File as python file in .py extension and use that py file in the frontend.

In [4]:
#Here the files is saved as crypto7 in same directory
import crypto8

# # # Provide the Date and Coin values
Date = input("Enter Date: ")
Coin = input("Enter crypto coin type: ")



Enter Date: 2025-12-15
Enter crypto coin type: Litecoin


In [5]:
# Call the main function
import crypto8

results = crypto8.main(Date, Coin)

# Retrieve the predictions and trading decisions
predictions = results[0]
trading_decisions = results[1]

# Print the results
print("Predictions:", predictions)
print("Trading decisions:", trading_decisions)


Predictions: 105.10531245055972
Trading decisions: Hold
